In [13]:
!pip install -r requirements.txt -q

In [14]:
# import subprocess

# subprocess.run(["aws", "s3", "sync", "s3://treetracker-training-images/pilot_annotations/PlantVillage/", "local_data/"])

In [15]:
import tqdm 
import network
import utils
import os
import random
import argparse
import numpy as np

from torch.utils import data
from datasets import LeafDataset
from utils import ext_transforms as et
from metrics import StreamSegMetrics, BinarySegMetrics

import torch
import torch.nn as nn
from utils.visualizer import Visualizer

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from torchvision import transforms
import matplotlib.pyplot as plt

import uuid



In [ ]:
!python3 binary_leaf_main.py --model deeplabv3plus_mobilenet --custom_data_path "local_data/" --batch_size 32 --lr 0.10 --total_itrs 700 --loss_type binary_dice --val_interval 50  --separable_conv --s3_model_dir "s3://treetracker-ml-models/sagemaker-training/"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
sagemaker.config INFO - Fetched defaults config from location: /home/ec2-user/SageMaker/treetracker-machine-learning/segmentation
INFO:sagemaker.experiments.run:The run (test) under experiment (main) already exists. Loading it.
Device: cpu
Calling s3 sync! This may take a while
Finished split
7671
7672
Root : local_data/
CWD:  /home/ec2-user/SageMaker/treetracker-machine-learning/segmentation
Children:  ['test.txt', 'val.txt', 'train.txt', '.ipynb_checkpoints']
Root : local_data/
CWD:  /home/ec2-user/Sag